## 1 Overview
This is an enviroment to trai and evaluate neural networks on learning logical calculi in a propositional logic. First the needed files are cloned from GitHub for it to run.

In [ ]:
# For Google Collab: Get repository and go to it in collab.
!git clone https://github.com/stereifberger/master-s-thesis
%cd master-s-thesis/

In [ ]:
# For Google Colab if above does not move to right directory
%cd /content/master-s-thesis/

In [ ]:
# For Jupyter Notebook (also in VSCode) after starting Jupyter server: Go to right directory.
%cd master-s-thesis/

In [ ]:
# Install required dependencies - not necessary on Google Colab
!pip install -r requirements.txt

In [ ]:
# Import required libraries from imports.py
from imports import *

## 2 Create dataset
First the dataset for training is generated. For this the function "create_dataset" from "generation.py" utilizes the functions "gen_outp_PA" to generate a set of random starting formulas, for which iterativly the applicability of rules is checked. All applicable rules are then used to generate new derivations. In each iteration of gen_oupt_PA, set by the iterations variable, new, longer examples are generated. get_conclusions generates some random conclusions using approximately the same procedure. The set of derivations is then filtered down to ones only containing those.

**Rules.** The rules are defined in calculi.py. Two sets are avaiable: Intuitionistic propositional logic (set below via "calculus = ipl") and classical propositional logic (set below via "calculus = cl").

**Dataset entries.**
- **x_trai.** traiing input: [INDEX, PREMISES, CONCLUSION]
- **y_trai_ordered.** Dataset of correct derivations where each sublist i correspnds to INDEX: [DERIVATIONS_0...DERIVATION_N]

**Encoding.** Propositional variables and logical constants are encoded as integers. The integers are then one-hot-encoded into unique sequences containing only 0s and ones with the length of the maximum integer value, the feature length. The shape of the individual entries is 2D: [SEQUENCE LENGTH, FEATURE LENGTH].

**Example entries withouth numerical representation and one-hot-encoding.**
- **x_trai.** [2345, A, A THEN B, B OR C]
- **y_trai_ordered.** Sublist 2345 is entry entry: [[A, A THEN B, B, B OR C], [A, A THEN B, B, A AND B, B OR C]]


In [ ]:
# Define input (as two and three dimensional vector) and output datset as outputs of generation.create_datset
# It applies rules of classical logic up to two times to do so on randomly generated premises
x_trai_2d, x_trai_3d, y_trai_ordered, max_y_trai_len = generation.create_dataset(iterations = [1,2], calculus = calculi.cl)

In [ ]:
torch.save(x_trai_2d, 'x_trai_2d.pt')                 # Save two dimensional input dataset to backup file
torch.save(x_trai_3d, 'x_trai_3d.pt')                 # Save three dimensional input dataset to backup file
torch.save(y_trai_ordered, 'y_trai_ordered.pt')       # Save output dataset to backup file
with open('Medium_max_y_trai_len.json', 'w') as file: # Save number with maximum length of derivations in output dataset to backup file
    json.dump(max_y_trai_len, file)

## 3 Prepare dataset and define models for traiing
Next with pytorch's dataloader the single traiing entries in x_trai are assigned to batches of size "batch size" in mixed order. Then the different models are defined using definitions from "architectures.py". These models are:

- Feedforward network (net)
- Recurrent neural network (RNNNet)
- Long-short-term memory (LSTMNet)
- Transformers (TransformerModel)

In [ ]:
if torch.cuda.is_available():                                           # Empty avaiable GPU memory when GPU is present
    torch.cuda.empty_cache()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   # GPU is defined as 'device' when present for using it

In [ ]:
two_d_shape = x_trai_2d.shape               # Get the 2d traiing dataset's shape for the model definitions later
thr_d_shape = x_trai_3d.shape               # Get the 23 traiing dataset's shape for the model definitions later
max_y_length = int(max_y_trai_len/14)       # Define maximum length of derivations for non one-hot encoded dataset

In [ ]:
x = torch.argmax(x_trai_3d, dim=2)          # Reverse one-hot encoding for encoder-decoder models
x[:, 0] = x_trai_2d[:, 0]
x_trai_nu = x

In [ ]:
trai_si = int(0.8 * len(x_trai_2d))                                     # Set trai-test split to 80-20 [^1]
test_si = len(x_trai_2d) - trai_si
x_trai_2d, x_test_2d = random_split(x_trai_2d, [trai_si, test_si])      # Choose random derivations from datsets distributed by trai-test split
x_trai_3d, x_test_3d = random_split(x_trai_3d, [trai_si, test_si])
x_trai_nu, x_test_nu = random_split(x_trai_nu, [trai_si, test_si])

In [ ]:
trai_dl_2d = DataLoader(dataset = x_trai_2d, shuffle = True, batch_size = 16)   # Create batches of size 16 from datasets
test_dl_2d = DataLoader(dataset = x_test_2d, shuffle = True, batch_size = 16)
trai_dl_3d = DataLoader(dataset = x_trai_3d, shuffle = True, batch_size = 16)
test_dl_3d = DataLoader(dataset = x_test_3d, shuffle = True, batch_size = 16)
trai_dl_nu = DataLoader(dataset = x_trai_nu, shuffle = True, batch_size = 64)
test_dl_nu = DataLoader(dataset = x_test_nu, shuffle = True, batch_size = 64)

In [ ]:
y_trai = y_trai_ordered.to(device)                      # Load ground truth data to GPU
y_trai_3d = y_trai.view(int(len(y_trai)),               # Reshape training 3d data
                        int(len(y_trai[0])),
                        int(len(y_trai[0][0])/14), 14)

For experiments the below parameters of the architectures have to be specified in accordance to which one is test.

In [ ]:
# Define the Encoder-Decoder networks
## FFN | Inputs: input_dim, hidden dim
encoder_ffn = architectures.Encoder_FFN(thr_d_shape[1], 150)
decoder_ffn = architectures.Decoder_FFN((max_y_length*14), 150)
ffn_ed_model = architectures.Seq2Seq(encoder_ffn, decoder_ffn, device)
## LSTM | Inputs: input_dim, embedding dim, hidden dim, nr layers, droput
encoder_lstm = architectures.Encoder_LSTM(thr_d_shape[1], 150, 150, 1, 0)
decoder_lstm = architectures.Decoder_LSTM(14, 150, 150, 3, 0)
lst_ed_model = architectures.Seq2Seq(encoder_lstm, decoder_lstm, device)
## Transformer-Encoder | Inputs:  input_dim, emb_dim, num_heads, hidden_dim, num_layers, dropout
encoder_tra = architectures.TransformerEncoder(thr_d_shape[1], 150, 5, 150, 1, dropout=0)
# Transformer-Decoder | Inputs: output_dim, emb_dim, num_heads, hidden_dim, num_layers
decoder_tra = architectures.TransformerDecoder(14, 150, 5, 150, 3)
tra_ed_model = architectures.Seq2SeqTransformer(encoder_tra, decoder_tra, device)

In [ ]:
lr = 0.001                                                              # Define Learning rate
ffn_ed_optimizer = torch.optim.AdamW(ffn_ed_model.parameters(),lr=lr)   # Define optimizers based on AdamW for models
lst_ed_optimizer = torch.optim.AdamW(lst_ed_model.parameters(),lr=lr)
tra_ed_optimizer = torch.optim.AdamW(tra_ed_model.parameters(),lr=lr)

# 4 Training

In [ ]:
criterion = nn.CrossEntropyLoss()

## 4.1 FFN Encoder-Decoder

In [ ]:
ffn_ed_model.to(device)                                                 # Load model to GPU

In [ ]:
# Tain model and save results
FFN_CELtrai, FFN_CELtest, FFN_ACCtrai, FFN_ACCtest = schedule.train_model(model = ffn_ed_model,
                                                                         dataloader_train = trai_dl_nu,
                                                                         dataloader_test = test_dl_nu,
                                                                         optimizer = ffn_ed_optimizer,
                                                                         criterion = criterion,
                                                                         epochs = 50,
                                                                         device = device,
                                                                         max_y_length = max_y_length,
                                                                         y_train = y_trai)
torch.save(ffn_ed_model.state_dict(), 'addition_model.pth')

In [ ]:
input, output = schedule.sanity(ffn_ed_model, test_dl_nu, device, max_y_length) # A sanity test for wheter the outputs look appropriate

In [ ]:
del ffn_ed_model            # Delete model from GPU to make space for new models
torch.cuda.empty_cache()    # Empty newly avaiable GPU memory

## 4.3 LSTM Encoder-Decoder

In [ ]:
lst_ed_model.to(device)     # Load model to GPU

In [ ]:
# Tain model and save results
LSTM_CELtrai, LSTM_CELtest, LSTM_ACCtrai, LSTM_ACCtest = schedule.train_model(model = lst_ed_model,
                                                                             dataloader_train = trai_dl_nu,
                                                                             dataloader_test = test_dl_nu,
                                                                             optimizer = lst_ed_optimizer,
                                                                             criterion = criterion,
                                                                             epochs = 50,
                                                                             device = device,
                                                                             max_y_length = max_y_length,
                                                                             y_train = y_trai_3d)
torch.save(lst_ed_model.state_dict(), 'addition_model.pth')     # Save model

In [ ]:
input, output = schedule.sanity(lst_ed_model, test_dl_nu, device, max_y_length) # A sanity test for wheter the outputs look appropriate

In [ ]:
del lst_ed_model            # Delete model from GPU to make space for new models
torch.cuda.empty_cache()    # Empt newly avaiable GPU memory

## 4.4 Transformer

In [ ]:
tra_ed_model.to(device)     # Load model to GPU

In [ ]:
TRA_CELtrai, TRA_CELtest, TRA_ACCtrai, TRA_ACCtest = schedule.train_model(model = tra_ed_model,
                                                                         dataloader_train = trai_dl_nu,
                                                                         dataloader_test = test_dl_nu,
                                                                         optimizer = tra_ed_optimizer,
                                                                         criterion = criterion,
                                                                         epochs = 50,
                                                                         device = device,
                                                                         max_y_length = max_y_length,
                                                                         y_train = y_trai_3d)
torch.save(tra_ed_model.state_dict(), 'addition_model.pth')

In [ ]:
schedule.sanity(tra_ed_model, test_dl_nu, device, max_y_length) # A sanity test for wheter the outputs look appropriate

In [ ]:
del tra_ed_model            # Delete model from GPU to make space for new models
torch.cuda.empty_cache()    # Empt newly avaiable GPU memory

## 5 Plot results
Here results from the preset parameters above can be plotted. Results in the thesis are plotted from csv files using tikz.

In [ ]:
# Feedforward Network,  Medium Dataset, Cross Entropy Loss
plt.figure(figsize=(5, 5), dpi=200)
x_data = list(range(50))
prop = fm.FontProperties(fname='/usr/share/fonts/opentype/freefont/FreeSerif.otf')
plt.plot(x_data, FFN_CELtrai, label='Training cross entropy loss')
plt.plot(x_data, FFN_CELtest, label='Test cross entropy loss')
plt.xlabel('Epochs')
plt.ylabel('Cross entropy loss')
plt.legend()
plt.show()

In [ ]:
# Feedforward Network, Medium Dataset, Accuracy
plt.figure(figsize=(5, 5), dpi=200)
x_data = list(range(50))
plt.plot(x_data, FFN_ACCtrai, label='Training accuracy')
plt.plot(x_data, FFN_ACCtest, label='Test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Long Short-Term Memory, Medium, Accuracy, Cross Entropy Loss
plt.figure(figsize=(5, 5), dpi=200)
x_data = list(range(50))
plt.plot(x_data, LSTM_CELtrai, label='Training cross entropy loss')
plt.plot(x_data, LSTM_CELtest, label='Test cross entropy loss')
plt.plot(x_data, LSTM_ACCtrai, label='Traiing accuracy')
plt.plot(x_data, LSTM_ACCtest, label='Test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Transformers results')
plt.legend()
plt.show()

In [ ]:
# Transformer, Medium, Accuracy, Cross Entropy Loss
plt.figure(figsize=(5, 5), dpi=200)
x_data = list(range(50))
plt.plot(x_data, TRA_CELtrai, label='Training cross entropy loss')
plt.plot(x_data, TRA_CELtest, label='Test cross entropy loss')
plt.plot(x_data, TRA_ACCtrai, label='Training accuracy')
plt.plot(x_data, TRA_ACCtest, label='Test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Transformers results')
plt.legend()
plt.show()

In [ ]:
# Test-Accuracy all Networks, Accuracy
plt.figure(figsize=(5, 5), dpi=200)
x_data = list(range(50))
#plt.plot(x_data, FFN_ACCtest, label='FFN Test accuracy')
plt.plot(x_data, RNN_ACCtest, label='RNN test accuracy')
plt.plot(x_data, RNN_ACCtest, label='RNN test accuracy')
plt.plot(x_data, LSTM_ACCtest, label='LSTM test accuracy')
plt.plot(x_data, TRA_ACCtest, label='Transformer test accuracy')
plt.xlabel('Epochs')
plt.ylabel('Transformers results')
plt.legend()
plt.show()

In [ ]:
# Test-Accuracy all Networks, Accuracy
plt.figure(figsize=(5, 5), dpi=200)
x_data = list(range(50))
#plt.plot(x_data, FFN_ACCtest, label='FFN Test accuracy')
plt.plot(x_data, RNN_CELtest, label='RNN test loss')
plt.plot(x_data, LSTM_CELtest, label='LSTM test loss')
plt.plot(x_data, TRA_CELtest, label='Transformer test loss')
plt.xlabel('Epochs')
plt.ylabel('Transformers results')
plt.legend()
plt.show()